In [1]:
import numpy as np
import torch
import tensorflow as tf

In [2]:
torch.set_default_dtype(torch.float64)
tf.keras.config.set_floatx('float64')

In [3]:
def add_to_class(Class):  
    """Register functions as methods in created class."""
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

# Dataset

## create dataset

In [4]:
from sklearn.datasets import make_classification

M: int = 100
N: int = 5
N_CLASS: int = 3

X, Y = make_classification(n_samples=M, n_features=N, n_classes=N_CLASS,
                           n_informative=N-1, n_redundant=0)

print(X.shape)
print(Y.shape)

(100, 5)
(100,)


## one hot encoding

In [5]:
Y_hat = torch.nn.functional.one_hot(torch.tensor(Y).long(), 3)
Y_hat.shape

torch.Size([100, 3])

## split dataset into train and valid

In [6]:
X_train, X_valid = torch.tensor(X[:85]), torch.tensor(X[85:])
Y_train, Y_valid = Y_hat[:85], Y_hat[85:]

print(X_train.shape, Y_train.shape)
print(X_valid.shape, Y_valid.shape)

torch.Size([85, 5]) torch.Size([85, 3])
torch.Size([15, 5]) torch.Size([15, 3])


# Model

## weights and bias

Trainables parameters
$$
\begin{align*}
\mathbf{W} &\in \mathbb{R}^{N \times Q} \\
\mathbf{b} &\in \mathbb{R}^{Q}
\end{align*}
$$
where $N$ is the number of features and $Q$ is the number of classes.

In [7]:
class SoftmaxClassifier:
    def __init__(self, n_features: int, n_classes: int):
        self.w = torch.randn(n_features, n_classes)
        self.b = torch.randn(n_classes)

    def copy_params(self, tf_model) -> None:
        """Copy the parameters from a TensorFlow model to this PyTorch model.

        Args:
            tf_model: A TensorFlow model from which to copy the parameters.

        Returns:
            None
        """
        self.w.copy_(torch.tensor(tf_model.weights[0].numpy()))
        self.b.copy_(torch.tensor(tf_model.weights[1].numpy()))

## weighted sum and softmax function

weighted sum
$$
\mathbf{Z}(\mathbf{X}) = \mathbf{X} \mathbf{W} + \mathbf{b} \\
\mathbf{Z} : \mathbb{R}^{M \times N} \rightarrow \mathbb{R}^{M \times Q}
$$

softmax function
$$
\sigma(\mathbf{Z}_{i,:})_{j} = \frac{\exp(\mathbf{Z}_{i,:})_{j}}
{\sum_{k=1}^{Q}(\exp(\mathbf{Z}_{i,:})_{k})}
$$
then:
$$
\sigma(\mathbf{Z}_{i,:}) = \begin{bmatrix}
    \sigma(\mathbf{Z}_{i,:})_{1} &
    \sigma(\mathbf{Z}_{i,:})_{2} &
    \cdots &
    \sigma(\mathbf{Z}_{i,:})_{Q}
\end{bmatrix}
$$
therefore:
$$
\sigma(\mathbf{Z}) = \begin{bmatrix}
    \sigma(\mathbf{Z}_{1,:}) \\
    \sigma(\mathbf{Z}_{2,:}) \\
    \vdots \\
    \sigma(\mathbf{Z}_{M,:})
\end{bmatrix} \\
\sigma(\mathbf{Z}) : \mathbb{R}^{M \times Q} \rightarrow \mathbb{R}^{M \times Q}
$$

In [8]:
@add_to_class(SoftmaxClassifier)
def predict(self, x: torch.Tensor) -> torch.Tensor:
    # weighted sum
    z = torch.matmul(x, self.w) + self.b
    # softmax function
    z_exp = torch.exp(z)
    y_pred = z_exp / z_exp.sum(1, keepdims=True)
    return y_pred

## CE

Loss function: Cross Entropy-loss:
$$
L(\mathbf{\hat{Y}}) = - \frac{1}{M} \sum_{i=1}^{M} \sum_{k=1}^{Q}(
    Y_{ik} \log(\hat{Y}_{ik})
) \\
L(\mathbf{\hat{Y}}) : \mathbb{R}^{M \times Q} \rightarrow \mathbb{R}
$$
**Remark**: for this case $\mathbf{\hat{Y}}$ is $\sigma(\mathbf{Z})$. It is not obligatory to use softmax for CE.<br>
Vectorized form:
$$
L(\mathbf{\hat{Y}}) = - \frac{1}{M} \sum_{i=1}^{M} \left(
    \mathbf{y}_{i,:}^T \log(\mathbf{\hat{y}}_{i,:})
\right)
$$
or
$$
L(\mathbf{\hat{Y}}) = - \frac{1}{M} \sum \left(
    \mathbf{Y} \odot \log(\mathbf{\hat{Y}})
\right)
$$

In [9]:
@add_to_class(SoftmaxClassifier)
def evaluate(self, x: torch.Tensor, y_true: torch.Tensor) -> float:
    y_pred = self.predict(x)
    loss = y_true * torch.log(y_pred)
    return - loss.sum().item() / len(y_true)

## Gradient

Gradient descent is:
$$
\frac{\partial L}{\partial \mathbf{W}} =
\frac{\partial L}{\partial \sigma}
\frac{\partial \sigma}{\partial \mathbf{Z}}
\frac{\partial \mathbf{Z}}{\partial \mathbf{W}}
$$
and
$$
\frac{\partial L}{\partial \mathbf{b}} =
\frac{\partial L}{\partial \sigma}
\frac{\partial \sigma}{\partial \mathbf{Z}}
\frac{\partial \mathbf{Z}}{\partial \mathbf{b}}
$$
where their shapes are:
$$
\begin{align*}
\frac{\partial L}{\partial \mathbf{W}} &\in \mathbb{R}^{N \times Q} \\
\frac{\partial L}{\partial \mathbf{b}} &\in \mathbb{R}^{Q} \\
\frac{\partial L}{\partial \sigma} &\in \mathbb{R}^{M \times Q} \\
\frac{\partial \sigma}{\partial \mathbf{Z}} &\in \mathbb{R}^{(M \times Q) \times (M \times Q)} \\
\frac{\partial \mathbf{Z}}{\partial \mathbf{W}} &\in \mathbb{R}^{(M \times Q) \times (N \times Q)} \\
\frac{\partial \mathbf{Z}}{\partial \mathbf{b}} &\in \mathbb{R}^{(M \times Q) \times Q}
\end{align*}
$$

### weighted sum derivative

#### $\frac{\partial \mathbf{Z}}{\partial \mathbf{W}}$
$$
\mathbf{X} = \begin{bmatrix}
    x_{11} & x_{12} & \cdots & x_{1N} \\
    x_{21} & x_{22} & \cdots & x_{2N} \\
    \vdots & \vdots & \ddots & \vdots \\
    x_{M1} & x_{M2} & \cdots & x_{MN}
\end{bmatrix} = \begin{bmatrix}
    \mathbf{x}_{1}^T \\
    \mathbf{x}_{2}^T \\
    \vdots \\
    \mathbf{x}_{M}^T
\end{bmatrix} \\
\mathbf{W} = \begin{bmatrix}
    w_{11} & w_{12} & \cdots & w_{1Q} \\
    w_{21} & w_{22} & \cdots & w_{2Q} \\
    \vdots & \vdots & \ddots & \vdots \\
    w_{N1} & w_{N2} & \cdots & w_{NQ} 
\end{bmatrix} = \begin{bmatrix}
    \mathbf{w}_{1} &
    \mathbf{w}_{2} &
    \cdots &
    \mathbf{w}_{Q}
\end{bmatrix}
$$
where $\mathbf{x}_{p}^T = \begin{bmatrix} x_{p1} & x_{p2} & \cdots & x_{pN} \end{bmatrix}$ 
and $\mathbf{w}_{q} = \begin{bmatrix} w_{1q} & w_{2q} & \cdots & w_{Nq} \end{bmatrix}^T$
, then
$$
\mathbf{XW} = \begin{bmatrix}
    \mathbf{x}_{1}^T \mathbf{w}_{1} & \mathbf{x}_{1}^T \mathbf{w}_{2} & \cdots & \mathbf{x}_{1}^T \mathbf{w}_{Q} \\
    \mathbf{x}_{2}^T \mathbf{w}_{1} & \mathbf{x}_{2}^T \mathbf{w}_{2} & \cdots & \mathbf{x}_{2}^T \mathbf{w}_{Q} \\
    \vdots & \vdots & \ddots & \vdots \\
    \mathbf{x}_{M}^T \mathbf{w}_{1} & \mathbf{x}_{M}^T \mathbf{w}_{2} & \cdots & \mathbf{x}_{M}^T \mathbf{w}_{Q}
\end{bmatrix}
$$
therefore:
$$
\frac{\partial \mathbf{XW}_{pq}}{\partial w_{ij}} = 
\frac{\partial \mathbf{x}_{p}^T\mathbf{w}_{q}}{\partial w_{ij}} = \begin{cases}
    x_{pi} & \text{ if } q=j \\ 
    0 & \text{ if } q\neq j 
\end{cases}
$$
for all $p = 1, ..., M$, $q, j = 1, ..., Q$ and $i = 1, ..., N$. <br>
Vectorized form:
$$
\frac{\partial \mathbf{Z}}{\partial \mathbf{W}} = 
\mathbb{I} \otimes \mathbf{X}
$$
where $\otimes$ is Kronecker product.

#### $\frac{\partial \mathbf{Z}}{\partial \mathbf{b}}$
$$
\mathbf{XW + b} = \begin{bmatrix}
    \mathbf{x}_{1}^T \mathbf{w}_{1}+b_1 & \mathbf{x}_{1}^T \mathbf{w}_{2}+b_2 & \cdots & \mathbf{x}_{1}^T \mathbf{w}_{Q}+b_Q \\
    \mathbf{x}_{2}^T \mathbf{w}_{1}+b_1 & \mathbf{x}_{2}^T \mathbf{w}_{2}+b_2 & \cdots & \mathbf{x}_{2}^T \mathbf{w}_{Q}+b_Q  \\
    \vdots & \vdots & \ddots & \vdots \\
    \mathbf{x}_{M}^T \mathbf{w}_{1}+b_1 & \mathbf{x}_{M}^T \mathbf{w}_{2}+b_2 & \cdots & \mathbf{x}_{M}^T \mathbf{w}_{Q}+b_Q 
\end{bmatrix}
$$
therefore:
$$
\frac{\partial \mathbf{(XW+b)}_{pq}}{\partial b_{i}} = 
\frac{\partial \mathbf{x}_{p}^T\mathbf{w}_{q} + b_{q}}{\partial b_{i}} = \begin{cases}
    1 & \text{ if } q=i \\ 
    0 & \text{ if } q\neq i
\end{cases}
$$
for all $p = 1, ..., M$ and $q,i = 1, ..., Q$.

### softmax derivative summary

First case:
$$
\frac{\partial \sigma(\mathbf{Z})_{p,:}}{\partial \mathbf{Z}_{i=p,:}} = \text{diag}(\sigma(\mathbf{Z}_{p,:})) - \sigma(\mathbf{Z}_{p,:}) \sigma(\mathbf{Z}_{p,:})^T
$$

Second case:
$$
\frac{\partial \sigma(\mathbf{Z})_{p,:}}{\partial \mathbf{Z}_{i\neq p,:}} = \mathbf{0}
$$
Please check [Softmax Function and Gradient](softmax_function_and_gradient.ipynb)

### cross-entropy derivative

We will use this CE:
$$
\begin{align*}
L(\mathbf{\hat{Y}}) &= - \frac{1}{M} \sum_{i=1}^{M} \left(
    \mathbf{y}_{i,:}^T \log(\mathbf{\hat{y}}_{i,:})
\right) \\
&= -\frac{1}{M} \left(
    \mathbf{y}_{1,:}^T \log(\mathbf{\hat{y}}_{1,:}) +
    \mathbf{y}_{2,:}^T \log(\mathbf{\hat{y}}_{2,:}) +
    ... + 
    \mathbf{y}_{M,:}^T \log(\mathbf{\hat{y}}_{M,:})
\right)
\end{align*}
$$
$$
\begin{align*}
\frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{y}}_{p,:}} &= \begin{bmatrix}
    \frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{y}}_{p1}} & 
    \frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{y}}_{p2}} &
    \cdots &
    \frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{y}}_{pQ}}
\end{bmatrix} \in \mathbb{R}^{1 \times Q} \\
\frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{Y}}} &= \begin{bmatrix}
    \frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{y}}_{1,:}} \\
    \frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{y}}_{2,:}} \\
    \vdots \\
    \frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{y}}_{M,:}}
\end{bmatrix} \in \mathbb{R}^{M \times Q}
\end{align*}
$$
then
$$
\begin{align*}
\frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{y}}_{p,:}} &=
-\frac{1}{M} \frac{\partial}{\partial \mathbf{\hat{y}}_{p,:}} \left(
    \mathbf{y}_{1,:}^T \log(\mathbf{\hat{y}}_{1,:}) +
    ... +
    \mathbf{y}_{p,:}^T \log(\mathbf{\hat{y}}_{p,:}) +
    ... + 
    \mathbf{y}_{M,:}^T \log(\mathbf{\hat{y}}_{M,:})
\right) \\
&= -\frac{1}{M} \frac{\partial}{\partial \mathbf{\hat{y}}_{p,:}} \left(
    \mathbf{y}_{p,:}^T \log(\mathbf{\hat{y}}_{p,:})
\right) \\
&= -\frac{1}{M} \left(
    \mathbf{y}_{p,:} \oslash \mathbf{\hat{y}}_{p,:}
\right)
\end{align*}
$$
where $\oslash$ is element-wise divide. <br>
Therefore
$$
\begin{align*}
\frac{\partial L(\mathbf{\hat{Y}})}{\partial \mathbf{\hat{Y}}} &= -\frac{1}{M} \begin{bmatrix}
    \mathbf{y}_{1,:} \oslash \mathbf{\hat{y}}_{1,:} \\
    \mathbf{y}_{2,:} \oslash \mathbf{\hat{y}}_{2,:} \\
    \vdots \\
    \mathbf{y}_{M,:} \oslash \mathbf{\hat{y}}_{M,:}
\end{bmatrix} \\
&= -\frac{1}{M} \mathbf{Y} \oslash \mathbf{\hat{Y}}
\end{align*}
$$

### pull it all together

$$
\begin{align*}
\frac{\partial L}{\partial \mathbf{Z}} &=
\frac{\partial L}{\partial \sigma}
\frac{\partial \sigma}{\partial \mathbf{Z}} \\
&\in \mathbb{R}^{{\color{Cyan} (M \times Q)} \times ({\color{Cyan} M \times Q} \times {\color{Orange} M \times Q})} \\
&\in \mathbb{R}^{\color{Orange} M \times Q}
\end{align*}
$$

#### $\frac{\partial L}{\partial \mathbf{W}}$
$$
\begin{align*}
\frac{\partial L}{\partial \mathbf{W}} &=
\frac{\partial L}{\partial \mathbf{Z}} 
\frac{\partial \mathbf{Z}}{\partial \mathbf{W}} \\
&\in \mathbb{R}^{{\color{Orange} (M \times Q)} \times ({\color{Orange} M \times Q} \times {\color{Magenta} N \times Q})} \\
&\in \mathbb{R}^{\color{Magenta} N \times Q}
\end{align*}
$$

#### $\frac{\partial L}{\partial \mathbf{b}}$
$$
\begin{align*}
\frac{\partial L}{\partial \mathbf{b}} &=
\frac{\partial L}{\partial \mathbf{Z}} 
\frac{\partial \mathbf{Z}}{\partial \mathbf{b}} \\
&\in \mathbb{R}^{{\color{Orange} (M \times Q)} \times ({\color{Orange} M \times Q} \times {\color{Magenta} Q})} \\
&\in \mathbb{R}^{\color{Magenta} Q}
\end{align*}
$$

#### Note
We can simply compute:
$$
\frac{\partial L}{\partial \mathbf{W}} = 
\frac{1}{M} \mathbf{X}^T \left(
    \sigma - \mathbf{Y}
\right)
$$
but the objective of this repository is to understand piece by piece and not simply give things for done.

In [10]:
@add_to_class(SoftmaxClassifier)
def update(self, x: torch.Tensor, y_true: torch.Tensor,
           y_pred: torch.Tensor, lr: float) -> None:
    m, n = x.shape
    m, n_classes = y_true.shape

    # cross entropy der
    delta = -(y_true / y_pred) / m

    # softmax der
    diag_a = torch.diag_embed(y_pred)
    outer_a = torch.einsum('ij,ik->ijk', y_pred, y_pred)
        
    sus_a = diag_a - outer_a
        
    soft_der = torch.zeros((m, n_classes, m, n_classes), dtype=y_pred.dtype)
    idx = torch.arange(m)
    soft_der[idx, :, idx, :] = sus_a
    delta = torch.einsum('pq,pqij->ij', delta, soft_der)

    # weighted sum der
    identity = torch.eye(n_classes)

    ## weight der
    w_der = torch.kron(x.unsqueeze(1).unsqueeze(3), identity.unsqueeze(0).unsqueeze(2))
    w_der = torch.einsum('pq,pqij->ij', delta, w_der)
    self.w -= lr * w_der

    ## bias der
    self.b -= lr * delta.sum(axis=0)

## Metrics

In [11]:
@add_to_class(SoftmaxClassifier)
def accuracy(self, y_true, y_pred):
    preds = y_pred.argmax(axis=-1)
    compare = (y_true.argmax(axis=-1) == preds).type(torch.float32)
    return compare.mean().item()

## Fit

In [12]:
@add_to_class(SoftmaxClassifier)
def fit(self, x_train: torch.Tensor, y_train: torch.Tensor, 
        epochs: int, lr: float, batch_size: int, 
        x_valid: torch.Tensor, y_valid: torch.Tensor) -> None:
    """fit the model using gradient descent.

    Args:
        x_train: Input tensor of shape (n_samples, num_features).
        y_train: Target tensor one hot of shape (n_samples, n_classes).
        epochs: Number of epochs to train.
        lr: learning rate (0, 1).
        batch_size: Int number of batch.
        x_valid: Input tensor of shape (n_valid_samples, num_features).
        y_valid: Input tensor one hot of shape (n_valid_samples, n_valid_classes).
    """
    for epoch in range(epochs):
        loss = 0
        num_batch = 0
        for batch in range(0, len(y_train), batch_size):
            num_batch += 1
            x_b = x_train[batch:batch+batch_size]
            y_b = y_train[batch:batch+batch_size]

            y_pred = self.predict(x_b)
            loss += self.evaluate(x_b, y_b)

            self.update(x_b, y_b, y_pred, lr)

        loss = round(loss / num_batch, 4)
        loss_v = round(self.evaluate(x_valid, y_valid), 4)
        acc = round(self.accuracy(y_valid, self.predict(x_valid)), 4)
        print(f'epoch: {epoch} - MSE: {loss} - MSE_v: {loss_v} - acc: {acc}')

# Scratch vs TF

## hyperparameters

In [13]:
LR = 0.01
EPOCHS = 16
BATCH = len(X_train) // 3

## models

### TF model

In [14]:
TFModel = tf.keras.Sequential([
    tf.keras.layers.Dense(units=N_CLASS, activation='softmax')
])

TFModel.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.SGD(learning_rate=LR),
    metrics = [tf.keras.metrics.Accuracy()]
)

TFModel.evaluate(X_train[:1], Y_train[:1])

TFModel.summary()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 0.5870   


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 3)              │            18 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18 (144.00 B)

 Trainable params: 18 (144.00 B)

 Non-trainable params: 0 (0.00 B)

### Scratch model

In [15]:
model = SoftmaxClassifier(N, N_CLASS)
model.copy_params(TFModel)

## comparison

In [16]:
def error(tensor_true, tensor_pred) -> float:
    """
     Calculates the percentage error between two tensors or floats.

     If the arguments are simple floats or ints, calculate the percentage error between them.
     If the arguments are Numpy ndarray and PyTorch tensor, calculate the percentage error between them.
     If the argumens are PyTorch tensors, calculate the percentage error between them.

     Args:
         tensor_true: The true tensor or true float.
         pred_tensor: The predicted tensor or the predicted float.

     Returns:
         The percentage error between the tensors or floats.
     """
    if isinstance(tensor_true, (float, int)) and isinstance(tensor_pred, (float, int)):
        return np.abs(tensor_true - tensor_pred) / np.abs(tensor_true) * 100
    elif type(tensor_true) is np.ndarray and type(tensor_pred) is torch.Tensor:
        e = np.abs(tensor_true - tensor_pred.numpy()) / np.abs(tensor_true)
        return np.mean(e) * 100
    e = torch.abs(tensor_true - tensor_pred) / torch.abs(tensor_true)
    return torch.mean(e) * 100

### predict

In [17]:
tf_predict = TFModel.predict(X_train, batch_size=len(X_train))
predict = model.predict(X_train)

error(tf_predict, predict)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step


1.3897221166873968e-14

### CE

In [18]:
tf_predict = TFModel.evaluate(X_train, Y_train, batch_size=len(X_train))[0]
predict = model.evaluate(X_train, Y_train)

error(tf_predict, predict)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - loss: 1.4981


1.4821332404403658e-14

### fit

In [19]:
TFModel.fit(X_train, Y_train, batch_size=BATCH, epochs=EPOCHS,
            shuffle=False, validation_data=(X_valid, Y_valid))

Epoch 1/16
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.0000e+00 - loss: 1.4361 - val_accuracy: 0.0000e+00 - val_loss: 2.5554
Epoch 2/16
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0000e+00 - loss: 1.4017 - val_accuracy: 0.0000e+00 - val_loss: 2.5119
Epoch 3/16
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0000e+00 - loss: 1.3695 - val_accuracy: 0.0000e+00 - val_loss: 2.4709
Epoch 4/16
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0000e+00 - loss: 1.3394 - val_accuracy: 0.0000e+00 - val_loss: 2.4325
Epoch 5/16
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0000e+00 - loss: 1.3114 - val_accuracy: 0.0000e+00 - val_loss: 2.3964
Epoch 6/16
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0000e+00 - loss: 1.2852 - val_accuracy: 0.0000e+00 - val_loss: 2.3625
Epoch 7/16
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0000e+00 - loss: 1.2609 - val_accuracy: 0.0000e+00 - val_loss: 2.3308
Epoch 8/16
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0000e+

In [20]:
TFModel.history.history['loss']

[1.5113473177242127,
 1.4588145850299226,
 1.4099029469181386,
 1.3645286729260422,
 1.3225660314128795,
 1.2838546242105078,
 1.2482081432709895,
 1.2154234432149613,
 1.1852890086294243,
 1.157592202953437,
 1.132125010540958,
 1.1086882488277352,
 1.0870944005342684,
 1.0671692996326831,
 1.0487529223319147,
 1.0316995130502224]

In [21]:
model.fit(X_train, Y_train, EPOCHS, LR, BATCH, X_valid, Y_valid)

epoch: 0 - MSE: 1.5113 - MSE_v: 2.5554 - acc: 0.2
epoch: 1 - MSE: 1.4588 - MSE_v: 2.5119 - acc: 0.2
epoch: 2 - MSE: 1.4099 - MSE_v: 2.4709 - acc: 0.2
epoch: 3 - MSE: 1.3645 - MSE_v: 2.4325 - acc: 0.2
epoch: 4 - MSE: 1.3226 - MSE_v: 2.3964 - acc: 0.3333
epoch: 5 - MSE: 1.2839 - MSE_v: 2.3625 - acc: 0.3333
epoch: 6 - MSE: 1.2482 - MSE_v: 2.3308 - acc: 0.3333
epoch: 7 - MSE: 1.2154 - MSE_v: 2.3009 - acc: 0.3333
epoch: 8 - MSE: 1.1853 - MSE_v: 2.2728 - acc: 0.3333
epoch: 9 - MSE: 1.1576 - MSE_v: 2.2464 - acc: 0.3333
epoch: 10 - MSE: 1.1321 - MSE_v: 2.2214 - acc: 0.3333
epoch: 11 - MSE: 1.1087 - MSE_v: 2.1978 - acc: 0.3333
epoch: 12 - MSE: 1.0871 - MSE_v: 2.1754 - acc: 0.3333
epoch: 13 - MSE: 1.0672 - MSE_v: 2.1542 - acc: 0.3333
epoch: 14 - MSE: 1.0488 - MSE_v: 2.1341 - acc: 0.3333
epoch: 15 - MSE: 1.0317 - MSE_v: 2.1148 - acc: 0.3333


In [22]:
tf_predict = TFModel.weights[0].numpy()
predict = model.w

error(tf_predict, predict)

1.7911208798267837e-13

In [23]:
tf_predict = TFModel.weights[1].numpy()
predict = model.b

error(tf_predict, predict)

1.522590863153646e-14

# Full train

In [24]:
model2 = SoftmaxClassifier(N, N_CLASS)

print(model2.b)
print(model2.w)

tensor([ 1.2648, -1.0024, -0.5563])
tensor([[-0.0956,  0.9671, -1.0026],
        [-0.8430,  0.7695, -0.9093],
        [-0.3887, -2.2423,  1.4185],
        [ 0.7278, -0.4257, -0.2401],
        [ 0.7755, -0.8383, -0.7018]])


In [25]:
model2.evaluate(X_valid, Y_valid)

5.505723176395554

In [26]:
model2.fit(X_train, Y_train, 350, 0.001, 1, X_valid, Y_valid)

epoch: 0 - MSE: 4.4024 - MSE_v: 5.3508 - acc: 0.2
epoch: 1 - MSE: 4.2623 - MSE_v: 5.1971 - acc: 0.2
epoch: 2 - MSE: 4.1249 - MSE_v: 5.0448 - acc: 0.2
epoch: 3 - MSE: 3.9904 - MSE_v: 4.8941 - acc: 0.2
epoch: 4 - MSE: 3.8588 - MSE_v: 4.745 - acc: 0.2
epoch: 5 - MSE: 3.7305 - MSE_v: 4.5978 - acc: 0.2
epoch: 6 - MSE: 3.6054 - MSE_v: 4.4527 - acc: 0.2
epoch: 7 - MSE: 3.4837 - MSE_v: 4.3099 - acc: 0.2
epoch: 8 - MSE: 3.3655 - MSE_v: 4.1696 - acc: 0.2
epoch: 9 - MSE: 3.2508 - MSE_v: 4.032 - acc: 0.2
epoch: 10 - MSE: 3.1397 - MSE_v: 3.8974 - acc: 0.2
epoch: 11 - MSE: 3.0321 - MSE_v: 3.7659 - acc: 0.2
epoch: 12 - MSE: 2.9281 - MSE_v: 3.6377 - acc: 0.2
epoch: 13 - MSE: 2.8275 - MSE_v: 3.513 - acc: 0.2
epoch: 14 - MSE: 2.7304 - MSE_v: 3.3919 - acc: 0.2
epoch: 15 - MSE: 2.6367 - MSE_v: 3.2743 - acc: 0.2
epoch: 16 - MSE: 2.5462 - MSE_v: 3.1605 - acc: 0.1333
epoch: 17 - MSE: 2.459 - MSE_v: 3.0505 - acc: 0.1333
epoch: 18 - MSE: 2.375 - MSE_v: 2.9441 - acc: 0.1333
epoch: 19 - MSE: 2.294 - MSE_v: 2.841

In [27]:
pred = model2.predict(X_valid)
pred.argmax(axis=-1)

tensor([0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 1, 1, 0, 1, 2])

In [28]:
Y_valid.argmax(axis=-1)

tensor([2, 2, 0, 1, 0, 0, 2, 1, 0, 0, 2, 2, 1, 1, 0])

In [29]:
model2.accuracy(Y_valid, pred)

0.46666666865348816